### N-Gram Language Model



This notebook implements an N-gram language model using a transformer architecture. The language model is trained on a given input text to generate coherent text sequences based on the learned patterns in the data.

# Dataset
The input text is read from a file named "input.txt". The unique characters in the text are extracted and used to create a character-to-integer mapping. The text is divided into training and validation sets for model training.

# Model Architecture
The model architecture is based on the transformer model. It consists of several components:

* Head: Represents one head of self-attention. It takes an input tensor and computes attention scores and weighted aggregations of values.
* MultiHeadAttention: Combines multiple heads of self-attention in parallel. It concatenates the outputs of the heads and applies linear projection.
* CustomGELU: Customized GELU activation function.
FeedForward: A simple linear layer followed by a non-linearity.
* Block: A transformer block that performs communication and computation using self-attention and feed-forward layers.
* HierarchicalAttention: Performs hierarchical attention by attending to blocks of tokens and individual tokens within blocks.
* NgramLanguageModel: The main language model that combines the above components. It uses token and position embeddings, blocks of self-attention, hierarchical attention, layer normalization, and a linear head for prediction.

# Training Loop
The model is trained using the Adam optimizer. The training loop consists of the following steps:

* Sample a batch of data from the training set.
* Evaluate the loss and perform backpropagation to update the model parameters.
* Every eval_interval iterations, evaluate the loss on both the training and  
  validation sets using the estimate_loss() function.
* Print the training and validation losses at regular intervals.

# Hyperparameters
* batch_size: Number of independent sequences processed in parallel.
* block_size: Maximum context length for predictions.
* max_iters: Maximum number of training iterations.
* eval_interval: Interval for evaluating the loss on training and validation sets.
* learning_rate: Learning rate for the optimizer.
* device: Device to run the computations on (CPU or CUDA).
* eval_iters: Number of iterations for estimating the loss during evaluation.
* n_embd: Embedding dimension.
* n_head: Number of heads for self-attention.
* n_layer: Number of transformer blocks.
* dropout: Dropout rate for regularization.
* ngram: Size of the N-gram context for generating new tokens.
* block_num_heads: Number of heads used in the hierarchical attention mechanism
  for attending to blocks of tokens.
* block_head_size: Size of each head in the block-level attention.
* token_num_heads: Number of heads used in the hierarchical attention mechanism
  for attending to individual tokens within blocks.
* token_head_size: Size of each head in the token-level attention.

# Usage
* Prepare the input text file named "input.txt".
* Set the desired hyperparameter values.
* Run the notebook to train the N-gram language model.
* The training progress, including the training and validation losses, will be
  printed at regular intervals.

Note: It's recommended to run this code on a GPU if available to speed up the training process.

In [6]:
# This code is based on code from https://github.com/karpathy/ng-video-lecture
# Link to original code: https://www.youtube.com/watch?v=kCc8FmEb1nY


In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math

# hyperparameters
batch_size = 64  # how many independent sequences will we process in parallel?
block_size = 128  # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 100
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 1024
n_head = 16
n_layer = 16
dropout = 0.3
# ------------

torch.manual_seed(1337)
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B,T,C)
        q = self.query(x)  # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C ** -0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embd)  # Adjusted dimensions here
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(out)
        out = self.proj(out)
        return out

class CustomGELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

import torch.nn.init as init

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            CustomGELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

        # Apply Xavier initialization to the linear layers
        for layer in self.net:
            if isinstance(layer, nn.Linear):
                init.xavier_uniform_(layer.weight)

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class HierarchicalAttention(nn.Module):
    def __init__(self, block_num_heads, block_head_size, token_num_heads, token_head_size):
        super().__init__()
        self.block_attention = MultiHeadAttention(block_num_heads, block_head_size)
        self.token_attention = MultiHeadAttention(token_num_heads, token_head_size)
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        block_out = self.block_attention(x)  # Attend to blocks of tokens
        x = x + self.dropout(block_out)
        token_out = self.token_attention(x)  # Attend to individual tokens within blocks
        x = x + self.dropout(token_out)
        x = self.proj(x)
        return x

class NgramLanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embd, block_size, n_head, n_layer, ngram, block_num_heads, block_head_size, token_num_heads, token_head_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.ngram = ngram
        self.hierarchical_attention = HierarchicalAttention(block_num_heads, block_head_size, token_num_heads, token_head_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.hierarchical_attention(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -self.ngram:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

ngram = 10
block_num_heads = 16
block_head_size = 512
token_num_heads = 8
token_head_size = 1024

model = NgramLanguageModel(vocab_size, n_embd, block_size, n_head, n_layer, ngram, block_num_heads, block_head_size, token_num_heads, token_head_size)
model.blocks = nn.Sequential(HierarchicalAttention(block_num_heads, block_head_size, token_num_heads, token_head_size))
m = model.to(device)

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


136.566839 M parameters
step 0: train loss 4.1314, val loss 4.1265
step 100: train loss 2.2920, val loss 2.2922
step 200: train loss 1.9330, val loss 1.9247
step 300: train loss 1.7699, val loss 1.7537
step 400: train loss 1.6668, val loss 1.6401
step 500: train loss 1.5849, val loss 1.5587
step 600: train loss 1.5397, val loss 1.5132
step 700: train loss 1.4986, val loss 1.4790
step 800: train loss 1.4560, val loss 1.4442
step 900: train loss 1.4300, val loss 1.4217
step 1000: train loss 1.4132, val loss 1.4059
step 1100: train loss 1.3943, val loss 1.3851
step 1200: train loss 1.3766, val loss 1.3761
step 1300: train loss 1.3640, val loss 1.3680
step 1400: train loss 1.3498, val loss 1.3543
step 1500: train loss 1.3396, val loss 1.3501
step 1600: train loss 1.3374, val loss 1.3442
step 1700: train loss 1.3183, val loss 1.3379
step 1800: train loss 1.3092, val loss 1.3237
step 1900: train loss 1.3041, val loss 1.3272
step 2000: train loss 1.2931, val loss 1.3164
step 2100: train loss 

In [4]:
context = torch.zeros((1, ngram), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=3000)[0].tolist()))












nelong to the palace the forth evext that it Jinn which the building thy hindment a piece of the Wezeer to this She kissmoryThus was the But
the purse in heav and capting fears are period five by such But the eyes the noret
camelO ped the his wayBarain This este regates with the unnels in that if than time pisting The law is disses divinaned that which She answered with the kinds upll severalous
eyes but the
sage DZam chanted
omi graam languided anothen carries with respecting the same of inate bridegrols them had eated on the policrouled by EElJabndanlness in it the cageles of who have no the slave away the that the time of Palace rective for respeating till dend head
his fteent which I ridised every inder the sup is place pasrothed his powen deprinciption of the city called by God
felighted out madays fir bath to be of the Khaleefeh opening the piece me stuffs but God Hatten Hone shearing
fish childbe liberation enter performs serpent and rhe liad of the veil Jinn arm tishn